# Data preparation

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!cp drive/MyDrive/qa_data.jsonl.zip .

In [25]:
!unzip qa_data.jsonl.zip

Archive:  qa_data.jsonl.zip
  inflating: qa_data.jsonl           
  inflating: __MACOSX/._qa_data.jsonl  


In [30]:
!head -n 200000 qa_data.jsonl > train.jsonl
!head -n 250000 qa_data.jsonl | tail -n 50000 > valid.jsonl
!head -n 300000 qa_data.jsonl | tail -n 50000 > test.jsonl

In [52]:
!pip install youtokentome

     |████████████████████████████████| 1.7 MB 5.3 MB/s 


In [96]:
!head -n 500000 qa_data.jsonl | sed 's/[^0-9а-яА-Я \-\.\?]//g' | sed 's/  / /g' > forbpe.txt

# Imports

In [ ]:
import pandas as pd
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

In [42]:
from torchtext.legacy.data import BucketIterator

In [53]:
import youtokentome as yttm

# Подготовка

In [98]:
%%time
vocab_size = 30_000
model_path = 'pretrained_bpe_lm.model'
yttm.BPE.train(data='forbpe.txt', vocab_size=vocab_size, model=model_path)

CPU times: user 17.9 s, sys: 3.06 s, total: 20.9 s
Wall time: 12.8 s


In [99]:
tokenizer = yttm.BPE(model=model_path)

In [101]:
UNK_TOKEN = '<UNK>'
START_TOKEN = tokenizer.vocab()[2]
END_TOKEN = tokenizer.vocab()[3]